In [16]:
import pandas as pd
import numpy as np
import math
import os
from tqdm import tqdm
from matplotlib import pyplot as plt
from PyRadioLoc.Utils.GeoUtils import GeoUtils
%run UtilsLightGBMFingerPrinting.ipynb
%run Imports.ipynb

# gera os resultados teoricos
for i in range(0,5):
    FILES = [
        (3*i + 13,'./k-fold-SVR/Test-SVR-Outdoor-Fold'+str(i+1)+'.csv'), # Usar CDB_20_SVR_Complete_k-fold 
        (3*i + 14,'./k-fold-SVR/Indoor_Test.csv'), # Usar CDB_20_SVR_Complete_k-fold
        (3*i + 15,'./k-fold-SVR/Test-SVR-Indoor-Outdoor-Fold'+str(i+1)+'.csv') # Usar CDB_20_SVR_Complete_k-fold
        ]
    TEST_DATABASE = "./k-fold-SVR/CDB-Fold"+str(i+1)+"-SVR.csv"
    generate_theorical_results(FILES, TEST_DATABASE)

FILES = [
    
    #k = 1 SVR
    './FinalResult/Resultados_Metodo_13.csv',  # CDB gerado com SVR dados Outdoor
    './FinalResult/Resultados_Metodo_14.csv',  # CDB gerado com SVR dados Indoor
    './FinalResult/Resultados_Metodo_15.csv',  # CDB gerado com SVR dados Indoor-Outdoor
    
    #k = 2 SVR
    './FinalResult/Resultados_Metodo_16.csv',  # CDB gerado com SVR dados Outdoor
    './FinalResult/Resultados_Metodo_17.csv',  # CDB gerado com SVR dados Indoor
    './FinalResult/Resultados_Metodo_18.csv',  # CDB gerado com SVR dados Indoor-Outdoor
    
    #k = 3 SVR
    './FinalResult/Resultados_Metodo_19.csv',  # CDB gerado com SVR dados Outdoor
    './FinalResult/Resultados_Metodo_20.csv',  # CDB gerado com SVR dados Indoor
    './FinalResult/Resultados_Metodo_21.csv',  # CDB gerado com SVR dados Indoor-Outdoor
    
    #k = 4 SVR
    './FinalResult/Resultados_Metodo_22.csv',  # CDB gerado com SVR dados Outdoor
    './FinalResult/Resultados_Metodo_23.csv',  # CDB gerado com SVR dados Indoor
    './FinalResult/Resultados_Metodo_24.csv',  # CDB gerado com SVR dados Indoor-Outdoor
    
    #k = 5 SVR
    './FinalResult/Resultados_Metodo_25.csv',  # CDB gerado com SVR dados Outdoor
    './FinalResult/Resultados_Metodo_26.csv',  # CDB gerado com SVR dados Indoor
    './FinalResult/Resultados_Metodo_27.csv',  # CDB gerado com SVR dados Indoor-Outdoor
    
    
]
METHODS = [x.split("_")[-1].split(".")[0] for x in FILES]     # lista contendo os metodos que serão usados
db = pd.read_csv('databases/meds_pandas.csv')
erbs = pd.read_csv('databases/Bts.csv')
#errorLocation = pd.read_csv('result/erros.csv')
ERROS_CSV_FILE = './result-k-fold/erros-SVR-k-fold.csv' 

400it [00:13, 29.78it/s]
400it [00:12, 32.28it/s]
400it [00:13, 30.22it/s]
400it [00:13, 29.61it/s]
400it [00:12, 33.10it/s]
400it [00:12, 29.09it/s]
400it [00:13, 26.60it/s]
400it [00:12, 32.60it/s]
400it [00:22, 11.95it/s]
400it [00:14, 28.43it/s]
400it [00:13, 30.18it/s]
400it [00:12, 32.60it/s]
400it [00:12, 30.97it/s]
400it [00:11, 35.33it/s]
400it [00:12, 33.27it/s]


In [17]:
# Gera o dataframe contendo os erros
errors_df = calculate_theorical_errors()
round(errors_df.describe(), 2)

,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
count,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00
mean,32.86,92.52,64.37,30.89,72.01,53.44,31.70,92.60,65.56,33.25,89.54,62.44,33.82,89.68,60.22
std,30.15,23.08,41.39,29.25,34.19,38.94,33.21,14.80,37.86,33.95,19.50,38.53,29.81,19.90,38.82
min,0.81,32.13,1.19,0.84,17.44,2.55,0.44,72.88,0.86,0.74,32.13,1.47,0.65,32.13,2.31
25%,11.05,81.69,19.86,10.48,42.13,17.44,9.23,83.07,25.98,9.28,81.69,20.98,12.02,81.69,19.60
50%,22.77,87.04,72.88,19.38,86.13,42.13,18.78,88.28,76.89,22.25,88.28,72.88,24.66,88.28,72.88
75%,45.00,104.99,102.64,42.56,92.82,88.28,44.01,104.36,92.82,44.76,104.36,92.82,46.88,104.36,92.82
max,177.81,137.37,173.53,161.79,124.19,199.50,243.81,124.19,160.97,213.93,124.19,152.23,165.70,137.37,129.69


In [18]:
test_erro = round(errors_df.describe(), 2)
test_erros = errors_df

In [19]:
std_outdoor = (test_erros['13'].std() + test_erros['16'].std() + test_erros['19'].std() + test_erros['22'].std() + test_erros['25'].std())/5
std_outdoor

31.271903090931893

In [20]:
std_indoor = (test_erros['14'].std() + test_erros['17'].std() + test_erros['20'].std() + test_erros['23'].std() + test_erros['26'].std())/5
std_indoor

22.292816332559035

In [21]:
std_indoor_outdoor = (test_erros['15'].std() + test_erros['18'].std() + test_erros['21'].std() + test_erros['24'].std() + test_erros['27'].std())/5
std_indoor_outdoor

39.11011016993733

# SVR-Outdoor:

In [22]:
test_erro = round(errors_df.describe(), 2)
test_erros = errors_df
mean_outdoor = (test_erros['13'] + test_erros['16'] + test_erros['19'] + test_erros['22'] + test_erros['25'])/5
mean_outdoor.head()

0    41.612
1    26.980
2    32.918
3    25.460
4    53.584
dtype: float64

In [23]:
min_outdoor = min(test_erro['13']['min'], test_erro['16']['min'], test_erro['19']['min'], test_erro['22']['min'], test_erro['25']['min'])
min_outdoor

0.44

In [24]:
max_outdoor = max(test_erro['13']['max'], test_erro['16']['max'], test_erro['19']['max'], test_erro['22']['max'], test_erro['25']['max'])
max_outdoor

243.81

# SVR-Indoor:

In [25]:
mean_indoor = (test_erros['14'] + test_erros['17'] + test_erros['20'] + test_erros['23'] + test_erros['26'])/5
#mean_indoor.head()
min_indoor = min(test_erro['14']['min'], test_erro['17']['min'], test_erro['20']['min'], test_erro['23']['min'], test_erro['26']['min'])
#min_indoor
max_indoor = max(test_erro['14']['max'], test_erro['17']['max'], test_erro['20']['max'], test_erro['23']['max'], test_erro['26']['max'])
#max_indoor
mean_indoor_outdoor = (test_erros['15'] + test_erros['18'] + test_erros['21'] + test_erros['24'] + test_erros['27'])/5
#mean_indoor_outdoor.head()
min_indoor_outdoor = min(test_erro['15']['min'], test_erro['18']['min'], test_erro['21']['min'], test_erro['24']['min'], test_erro['27']['min'])
#min_indoor_outdoor
max_indoor_outdoor = max(test_erro['15']['max'], test_erro['18']['max'], test_erro['21']['max'], test_erro['24']['max'], test_erro['27']['max'])
#max_indoor_outdoor
erros_SVR = [mean_outdoor,mean_indoor, mean_indoor_outdoor ]
erros_SVR = pd.DataFrame(erros_SVR)
erros_SVR = erros_SVR.T
erros_SVR = round(erros_SVR,2)
erros_SVR.to_csv(ERROS_CSV_FILE, index=False)
erros_SVR.head()


,0,1,2
0,41.61,88.03,21.59
1,26.98,50.32,27.78
2,32.92,96.72,49.56
3,25.46,61.79,45.17
4,53.58,96.85,51.71


In [26]:
max_indoor

137.37

In [27]:
min_indoor

17.44

In [28]:
max_indoor_outdoor

199.5

In [29]:
min_indoor_outdoor

0.86

In [30]:
results_SVR = round(erros_SVR.describe(), 2)
results_SVR
#results_SVR.to_csv('./result-k-fold/Result-SVR-k-fold.csv', index=False)

,0,1,2
count,400.00,400.00,400.00
mean,32.50,87.27,61.21
std,14.04,20.79,32.09
min,7.38,42.04,8.15
25%,21.87,71.78,33.08
50%,31.20,88.03,61.79
75%,41.41,103.00,88.03
max,83.07,126.83,126.83
